`ppdire.py` subpackage: examples
================================
Here, we will illustrate how to use `ppdire.py` to perform different types of projection pursuit dimension reduction. 

To run a toy example, start by sourcing packages and data: 

In [10]:
# Load data
import pandas as ps
import numpy as np
data = ps.read_csv("../data/Returns_shares.csv")

columns = data.columns[2:8]
(n,p) = data.shape
datav = np.matrix(data.values[:,2:8].astype('float64'))
y = datav[:,0]
X = datav[:,1:5]
        
# Scale data
from direpack import VersatileScaler
centring = VersatileScaler()
Xs = centring.fit_transform(X)

1\. Comparison of PP estimates to Scikit-Learn 
======================================
Let us at first run `ppdire` to produce slow, approximate PP estimates of 
PCA and PLS. This makes it easy to verify that the algorithm is correct. 
        
1\.1\. PCA
--------------
By setting the projection index to variance, projection pursuit is a slow, approximate way to calculate PCA. Let's compare the `ppdire` results to `sklearn`'s.

- PCA ex `scikit-learn` 

In [11]:
import sklearn.decomposition as skd
skpca = skd.PCA(n_components=4)

In [12]:
%time skpca.fit(Xs)
skpca.components_.T # sklearn outputs loadings as rows ! 

CPU times: user 1.94 ms, sys: 1.27 ms, total: 3.21 ms
Wall time: 8.02 ms


array([[ 0.6160923 ,  0.09574799, -0.27553202,  0.73167254],
       [ 0.3612025 ,  0.79963754,  0.40566716, -0.25602091],
       [ 0.57369773, -0.2220554 , -0.47175995, -0.63166832],
       [ 0.40104108, -0.54963945,  0.73277473,  0.01018414]])

- PCA ex `ppdire`, using SLSQP optimization 

In [13]:
from direpack import dicomo, ppdire
pppca = ppdire(projection_index = dicomo, pi_arguments = {'mode' : 'var'}, n_components=4, optimizer='SLSQP')

In [14]:
%time pppca.fit(X)
pppca.x_loadings_

CPU times: user 208 ms, sys: 18.1 ms, total: 227 ms
Wall time: 143 ms


array([[-0.61607418,  0.09583099,  0.27564666, -0.73163593],
       [-0.36117418,  0.79968851, -0.40559672,  0.25604181],
       [-0.57366555, -0.22197832,  0.47186547,  0.63170265],
       [-0.40114045, -0.54958196, -0.73270266, -0.01015964]])

- PCA ex `ppdire`, using its native `grid` algorithm optimization \[1\].

In [16]:
pppca = ppdire(projection_index = dicomo, pi_arguments = {'mode' : 'var'}, n_components=4, optimizer='grid',optimizer_options={'ndir':1000,'maxiter':1000})

In [17]:
%time pppca.fit(X)
pppca.x_loadings_

1
2
3
1
2
3
4
1
2
3
4
1
2
CPU times: user 38.5 s, sys: 1.4 s, total: 39.9 s
Wall time: 37.9 s


array([[ 0.61598274, -0.09421448, -0.27613309,  0.73205157],
       [ 0.36137778, -0.80127483,  0.40173098, -0.25522632],
       [ 0.57377558,  0.22404487, -0.47051245, -0.63154901],
       [ 0.40094043,  0.54670882,  0.73551452,  0.01041449]])

1\.2\. PLS 
----------
Likewise, by setting the projection index to covariance, one obtains partial least squares. 

- PLS ex `scikit-learn`

In [ ]:
import sklearn.cross_decomposition as skc
skpls = skc.PLSRegression(n_components=4)
skpls.fit(Xs,(y-np.mean(y))/np.std(y))
skpls.x_scores_
print(skpls.coef_) 
Xs*skpls.coef_*np.std(y) + np.mean(y)

- PLS ex `ppdire`, using SLSQP optimization 

In [ ]:
pppls = ppdire(projection_index = dicomo, pi_arguments = {'mode' : 'cov'}, n_components=4, square_pi=True, optimizer='SLSQP', optimizer_options={'maxiter':500})
pppls.fit(X,y)
pppls.x_scores_
print(pppls.coef_scaled_) # Column 4 should agree with skpls.coef_
pppls.fitted_  

- PLS ex `ppdire`, `grid` optimization

In [ ]:
pppls = ppdire(projection_index = dicomo, pi_arguments = {'mode' : 'cov'}, n_components=4, square_pi=True, optimizer='grid',optimizer_options={'ndir':1000,'maxiter':1000})
pppls.fit(X,y)
pppls.x_scores_
print(pppls.coef_scaled_) # Column 4 should agree with skpls.coef_
pppls.fitted_  

Remark: Dimension Reduction techniques based on projection onto latent variables, 
such as PCA, PLS and ICA, are sign indeterminate with respect to the components. 
Therefore, signs of estimates by different algorithms can be opposed, yet the 
absolute values should be identical up to algorithm precision.  Here, this implies
that `sklearn` and `ppdire`'s `x_scores_` and `x_loadings` can have opposed signs,
yet the coefficients and fitted responses should be identical. 

2\. Robust projection pursuit estimators
=================================

Note that optimization through `scipy.optimize` is much more efficient than the native `grid` algorithm, yet will only provide correct results for classical projection indices. The native `grid` algorithm should be used when 
the projection index involves order statistics of any kind, such as ranks, trimming, winsorizing, or empirical quantiles.

- Robust PCA based on the Median Absolute Deviation (MAD) \[2\]. 

In [ ]:
lcpca = ppdire(projection_index = dicomo, pi_arguments = {'mode' : 'var', 'center': 'median'}, n_components=4, optimizer='grid',optimizer_options={'ndir':1000,'maxiter':10})
#set a higher maxiter for convergence and precision! 
lcpca.fit(X)
lcpca.x_loadings_

To extend to Robust PCR, just add `y`: 

In [ ]:
 lcpca.fit(X,y,regopt='robust')

- Robust Continuum Regression \[4\] based on trimmed continuum association: 

In [ ]:
rcr = ppdire(projection_index = dicomo, pi_arguments = {'mode' : 'continuum'}, n_components=4, trimming=.1, alpha=.5, optimizer='grid',optimizer_options={'ndir':250,'maxiter':1000})
rcr.fit(X,y=y,regopt='robust')
rcr.x_loadings_
rcr.x_scores_
rcr.coef_scaled_
rcr.predict(X[:2666])

Let us now plot the results. The `plot` subpackage contains a plotting function for `ppdire`. To plot predicted vs. actual values:

In [ ]:
from direpack import ppdire_plot
dr_plot=ppdire_plot(rcr,['w','w','g','y','m','b','k'])
dr_plot.plot_yyp(label='KMB',title='fitted vs true quotation, training set')

In [ ]:
dr_plot.plot_yyp(ytruev=y[2666:],Xn=X[2666:],label='KMB',title='fitted vs true quotation, test set')

To plot scores: 

In [ ]:
dr_plot.plot_projections(Xn=X[2666:],label='KMB',title='fitted vs true quotation, test set')

3\. Projection pursuit generalized betas
================================

Generalized betas are obtained as the projection pursuit weights using the 
co-moment analysis projection index (CAPI) \[2\]. 

In [ ]:
from ppdire import capi 
est = ppdire(projection_index = capi, pi_arguments = {'max_degree' : 3,'projection_index': dicomo, 'scaling': False}, n_components=1, trimming=0,center_data=True,scale_data=True)
est.fit(X,y=y,ndir=200)
est.x_weights_

Note that these data aren't the greatest illustration. Evaluating CAPI projections, makes more sense if y is a market index, e.g. SPX. 

4\. Cross-validating through `scikit-learn` 
===========================================

The `ppdire` class is 100% compatible with `scikit-learn`, which allows, for instance, hyperparameter tuning through `GridSearchCV`. 
To try out, uncomment the line below and run. (this may take some time). 

In [ ]:
# Uncomment to try out:

# from sklearn.model_selection import GridSearchCV
# rcr_cv = GridSearchCV(ppdire(projection_index=dicomo, 
#                                    pi_arguments = {'mode' : 'continuum'
#                                                    }, 
#                                    optimizer = 'grid',
#                                   optimizer_options = {'ndir':1000,'maxiter':1000}
#                                   ), 
#                             cv=10, 
#                             param_grid={"n_components": [1, 2, 3], 
#                                         "alpha": np.arange(.1,3,.3).tolist(),
#                                         "trimming": [0, .15]
#                                        }
#                            )
#       rcr_cv.fit(X[:2666],y[:2666]) 
#       rcr_cv.best_params_
#       rcr_cv.predict(X[2666:])

5\. Data compression
=================
While `ppdire` is very flexible and can project according to a very wide variety 
of projection indices, it can be computationally demanding. For flat data tables,
a workaround has been built in. However, not that running the code in the next field can take quite some time nonetheless.   

In [ ]:
# Load flat data 
datan = ps.read_csv("../data/Glass_df.csv")
X = datan.values[:,100:300]
y = datan.values[:,2]
        
# Now compare
rcr = ppdire(projection_index = dicomo, 
                    pi_arguments = {'mode' : 'continuum'}, 
                    n_components=4, 
                    trimming=.1, 
                    alpha=.5, 
                    compression = False, 
                    optimizer='grid',
                    optimizer_options={'ndir':1000,'maxiter':1000})
rcr.fit(X,y)
print(rcr.coef_)
        
rcr = ppdire(projection_index = dicomo, 
                    pi_arguments = {'mode' : 'continuum'}, 
                    n_components=4, 
                    trimming=.1, 
                    alpha=.5, 
                    compression = True, 
                    optimizer='grid',
                    optimizer_options={'ndir':1000,'maxiter':1000})
rcr.fit(X,y)
rcr.coef_

However, compression will not work properly if the data contain several low scale 
varables. In this example, it will not work for `X = datan.values[:,8:751]`. This 
will throw a warning, and `ppdire` will continue without compression. 

References
----------------
1. [Robust Multivariate Methods: The Projection Pursuit Approach](https://link.springer.com/chapter/10.1007/3-540-31314-1_32), Peter Filzmoser, Sven Serneels, Christophe Croux and Pierre J. Van Espen, in: From Data and Information Analysis to Knowledge Engineering, Spiliopoulou, M., Kruse, R., Borgelt, C., Nuernberger, A. and Gaul, W., eds., Springer Verlag, Berlin, Germany, 2006, pages 270--277.
2. [Projection pursuit based generalized betas accounting for higher order co-moment effects in financial market analysis](https://arxiv.org/pdf/1908.00141.pdf), Sven Serneels, in: JSM Proceedings, Business and Economic Statistics Section. Alexandria, VA: American Statistical Association, 2019, 3009-3035.
3. Robust principal components and dispersion matrices via projection pursuit, Chen, Z. and Li, G., Research Report, Department of Statistics, Harvard University, 1981.
4. [Robust Continuum Regression](https://www.sciencedirect.com/science/article/abs/pii/S0169743904002667), Sven Serneels, Peter Filzmoser, Christophe Croux, Pierre J. Van Espen, Chemometrics and Intelligent Laboratory Systems, 76 (2005), 197-204.